<a href="https://colab.research.google.com/github/rrwiren/ilmanlaatu-ennuste-helsinki/blob/main/PIPELINE_v0.6_XGGBoost.ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""
==========================================================================================
 PIPELINE v0.6 - XGBOOST-MALLIN KOULUTUS, EVALUOINTI JA PIIRTEIDEN TÄRKEYS
 (Early Stopping Poistettu, Korjattu Kutsu)
==========================================================================================

 Versio: 0.6 (Early Stopping Poistettu, Korjattu Kutsu)
 Päivitetty: 2025-04-12 13:32 (BST)
 Edellinen versio: 0.5 (Esikäsittely, EDA, Piirteet, Baseline)

 Skriptin tarkoitus:
 --------------------
 1. Ladata valmiiksi esikäsitelty ja piirteillä täydennetty data (v0.5).
 2. Valita mallinnuksessa käytettävät piirteet.
 3. Jakaa data opetus- ja testijoukkoihin.
 4. Kouluttaa XGBoost Regressor -malli ILMAN early stoppingia.
 5. Evaluoida mallin suorituskyky ja verrata baselineen.
 6. Analysoida ja visualisoida mallin piirteiden tärkeysjärjestys.
 7. (Valinnaisesti) Tallentaa koulutettu malli.

 Vaaditut kirjastot: <Samat kuin ennen>
"""

# === Vaihe 1: Alustus ja Kirjastojen Tuonti ===
print("="*60); print(" Vaihe 1: Alustus ja Kirjastojen Tuonti"); print("="*60)
import pandas as pd; import numpy as np; import requests; import io; import os
import matplotlib.pyplot as plt; import seaborn as sns;
import xgboost as xgb
# Huom: EI importata EarlyStopping callbackia, koska sitä ei käytetä
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib
plt.style.use('ggplot')
print(f"--- Käytössä oleva XGBoost-versio: {xgb.__version__} ---")

# --- Konfiguraatio ---
INPUT_DATA_URL = "https://raw.githubusercontent.com/rrwiren/ilmanlaatu-ennuste-helsinki/main/data/processed/Helsinki_Data_With_Features_Pipeline_v0.5.parquet"
print(f"Käytetään dataa URL-osoitteesta: {INPUT_DATA_URL}")
TARGET_VARIABLE = 'Ozone'; TEST_DATA_RATIO = 0.2; RANDOM_STATE = 42; SAVE_MODEL = True
MODEL_SAVE_PATH = "models/xgboost_v0.6_fi_no_es.joblib" # Edelleen sama nimi tälle versiolle
FEATURE_COLUMNS = ['Visibility','Temperature_Max','Temperature_Min','Temperature','WindSpeed','WindDirection','Pressure','PM10','PM25','NO2','NO','SO2','BC','hour','dayofweek','dayofyear','month','year','weekofyear','hour_sin','hour_cos','dayofweek_sin','dayofweek_cos','month_sin','month_cos']
print(f"\nMallinnuksessa käytettävät piirteet ({len(FEATURE_COLUMNS)} kpl): {FEATURE_COLUMNS}")
XGB_PARAMS = {'objective':'reg:squarederror','n_estimators':500,'learning_rate':0.05,'max_depth':6,'subsample':0.8,'colsample_bytree':0.8,'random_state':RANDOM_STATE,'n_jobs':-1,'tree_method':'hist','eval_metric':'rmse'}
# Early Stopping POISTETTU KÄYTÖSTÄ
# USE_EARLY_STOPPING = False # Tätä ei enää tarvita, koska logiikka poistettu funktiosta


# === Pipeline Vaihe 1: Datan Lataus ===
def lataa_esikasitelty_data(url):
    """Lataa esikäsitellyn datan Parquet-tiedostosta URL-osoitteesta."""
    print("\n" + "="*60); print(" Pipeline Vaihe 1: Datan Lataus"); print("="*60)
    print(f"Yritetään ladata dataa osoitteesta: {url}")
    df = None
    try: import pyarrow; print("'pyarrow' löytyy."); df = pd.read_parquet(url)
    except ImportError: print("VIRHE: 'pyarrow' puuttuu."); return None
    except Exception as e: print(f"VIRHE datan latauksessa: {e}"); return None
    if df is not None:
        print(f"Data ladattu."); print(f"Muoto: {df.shape}")
        if not isinstance(df.index, pd.DatetimeIndex): print("VAROITUS: Indeksi ei ole DatetimeIndex..."); df.index = pd.to_datetime(df.index)
        return df
    return None

# === Pipeline Vaihe 2: Datan Jako ===
def jaa_data(df, target_sarake, test_osuus):
    """Jakaa datan kronologisesti opetus- ja testijoukkoihin."""
    print("\n" + "="*60); print(" Pipeline Vaihe 2: Datan Jako"); print("="*60)
    if df is None or df.empty: print("VIRHE: Ei dataa."); return None, None, None, None
    if target_sarake not in df.columns: print(f"VIRHE: Kohde '{target_sarake}' puuttuu."); return None, None, None, None
    n=len(df); split_index=int(n*(1-test_osuus));
    if split_index<=0 or split_index>=n: print(f"VIRHE: Testikoko {test_osuus} virheellinen."); return None,None,None,None
    df_train=df.iloc[:split_index]; df_test=df.iloc[split_index:]
    print(f"Data jaettu: Opetus {df_train.shape[0]} ({df_train.index.min()} - {df_train.index.max()})")
    print(f"            Testi  {df_test.shape[0]} ({df_test.index.min()} - {df_test.index.max()})")
    y_train=df_train[target_sarake]; y_test=df_test[target_sarake]
    X_train=df_train.drop(columns=[target_sarake]); X_test=df_test.drop(columns=[target_sarake])
    print("Opetus/testijoukot luotu (X,y)."); return X_train, y_train, X_test, y_test

# === Pipeline Vaihe 3: Piirteiden Valinta ===
def valitse_piirteet(X_train, X_test, piirre_lista):
    """Valitsee määritellyt piirteet opetus- ja testidatasta."""
    print("\n" + "="*60); print(" Pipeline Vaihe 3: Piirteiden Valinta"); print("="*60)
    if X_train is None or X_test is None: print("VIRHE: Data puuttuu."); return None, None
    missing=[p for p in piirre_lista if p not in X_train.columns]
    if missing: print(f"VIRHE: Piirteitä puuttuu: {missing}."); return None, None
    print(f"Valitaan {len(piirre_lista)} piirrettä."); return X_train[piirre_lista], X_test[piirre_lista]

# === Pipeline Vaihe 4: XGBoost-mallin Koulutus ===
# MUOKATTU: Funktio ottaa nyt vastaan vain tarvittavat argumentit
def kouluta_xgboost_malli(X_train, y_train, xgb_parametrit):
    """Kouluttaa XGBoost Regressor -mallin ILMAN early stoppingia."""
    print("\n" + "="*60); print(" Pipeline Vaihe 4: XGBoost-mallin Koulutus"); print("="*60)
    if X_train is None or y_train is None: print("VIRHE: Koulutusdata puuttuu."); return None

    print("Käytettävät XGBoost-parametrit (alustuksessa):"); print(xgb_parametrit)
    # Alustetaan malli
    model = xgb.XGBRegressor(**xgb_parametrit)

    try:
        print("\nAloitetaan mallin koulutus...")
        # Koulutetaan ilman early stopping -parametreja .fit()-kutsussa
        model.fit(X_train, y_train)
        print("XGBoost-malli koulutettu onnistuneesti.")
        # Tulostetaan puiden määrä (pitäisi olla n_estimators)
        try:
            actual_n_estimators = len(model.get_booster().get_dump())
            print(f"Koulutettu malli sisältää {actual_n_estimators} puuta.")
        except: pass
        return model
    except Exception as e:
        print(f"VIRHE XGBoost-mallin koulutuksessa: {e}")
        import traceback; traceback.print_exc()
        if 'tree_method="gpu_hist"' in str(xgb_parametrit) and ('No GPU found' in str(e) or 'xgboost is not compiled with GPU support' in str(e)): print("-> Vinkki: GPU-kiihdytys ei käytettävissä.")
        return None

# === Pipeline Vaihe 5: Mallin Evaluointi ===
def evaluoi_malli(model, X, y, datasetti_nimi):
    """Evaluoi koulutetun mallin suorituskyvyn annetulla datajoukolla."""
    print("\n" + "="*60); print(f" Pipeline Vaihe 5: Mallin Evaluointi ({datasetti_nimi})"); print("="*60)
    if model is None or X is None or y is None: print("VIRHE: Virheelliset syötteet."); return None
    try:
        y_pred = model.predict(X); mae = mean_absolute_error(y, y_pred); rmse = np.sqrt(mean_squared_error(y, y_pred)); r2 = r2_score(y, y_pred)
        print(f"Suorituskykymetriikat ({datasetti_nimi}):"); print(f"  MAE:  {mae:.3f}"); print(f"  RMSE: {rmse:.3f}"); print(f"  R²:   {r2:.3f}")
        plt.figure(figsize=(18, 5)); plt.plot(y.index, y, label='Todelliset', alpha=0.8, color='blue'); plt.plot(y.index, y_pred, label='Ennusteet (XGBoost)', alpha=0.8, linestyle='--', color='red')
        plt.title(f'XGBoost Mallin Ennusteet vs. Todelliset ({datasetti_nimi})'); plt.xlabel('Aika'); plt.ylabel(TARGET_VARIABLE); plt.legend(); plt.grid(True); plt.tight_layout(); plt.show()
        return {'MAE': mae, 'RMSE': rmse, 'R2': r2}
    except Exception as e: print(f"VIRHE mallin evaluoinnissa ({datasetti_nimi}): {e}"); return None

# === Pipeline Vaihe 5b: Piirteiden Tärkeysanalyysi ===
def analysoi_piirteiden_tarkeys(model, X_train):
    """Analysoi ja visualisoi XGBoost-mallin piirteiden tärkeyden."""
    print("\n" + "="*60); print(" Pipeline Vaihe 5b: XGBoost Piirteiden Tärkeys"); print("="*60)
    if model is None or X_train is None: print("VIRHE: Malli tai opetusdata puuttuu."); return
    try:
        ax = xgb.plot_importance(model, max_num_features=20); ax.figure.set_size_inches(10, 10)
        plt.title('XGBoost Piirteiden Tärkeys (Gain / Paino)'); plt.tight_layout(); plt.show()
        feature_importance = pd.Series(model.feature_importances_, index=X_train.columns)
        top_features = feature_importance.nlargest(20)
        plt.figure(figsize=(10, 10)); sns.barplot(x=top_features.values, y=top_features.index)
        plt.title('Top 20 Tärkeintä Piirrettä (XGBoost Importance)'); plt.xlabel('Tärkeys'); plt.ylabel('Piirre'); plt.tight_layout(); plt.show()
        print("\nPiirteiden tärkeysjärjestys:"); print(feature_importance.sort_values(ascending=False).to_string())
    except Exception as fi_e: print(f"VIRHE piirteiden tärkeyden analysoinnissa: {fi_e}")

# === Pipeline Vaihe 6: Mallin Tallennus (Valinnainen) ===
def tallenna_malli(model, polku):
    """Tallentaa koulutetun mallin levylle käyttäen joblibia."""
    print("\n" + "="*60); print(" Pipeline Vaihe 6: Mallin Tallennus"); print("="*60)
    if model is None: print("VIRHE: Ei mallia tallennettavaksi."); return False
    try:
        output_folder = os.path.dirname(polku);
        if output_folder: os.makedirs(output_folder, exist_ok=True)
        joblib.dump(model, polku); print(f"Malli tallennettu: {polku}"); return True
    except Exception as e: print(f"VIRHE mallin tallennuksessa {polku}: {e}"); return False

# === Pääsuorituslohko ===
if __name__ == "__main__":
    print("\n" + "="*80); print(" ALOITETAAN XGBOOST PIPELINE v0.6 (NO Early Stopping - Korjattu Kutsu)"); print("="*80)
    df_data, model = None, None; X_train, y_train, X_test, y_test = None, None, None, None
    xgboost_results = None; tallennettu_ok = False;

    try:
        # Vaihe 1: Lataa data
        df_data = lataa_esikasitelty_data(INPUT_DATA_URL)
        if df_data is None: raise ValueError("Datan lataus epäonnistui.")
        # Vaihe 2: Jaa data
        X_train_all, y_train, X_test_all, y_test = jaa_data(df_data, TARGET_VARIABLE, test_osuus=TEST_DATA_RATIO)
        if X_train_all is None: raise ValueError("Datan jako epäonnistui.")
        # Vaihe 3: Valitse piirteet
        X_train, X_test = valitse_piirteet(X_train_all, X_test_all, FEATURE_COLUMNS)
        if X_train is None: raise ValueError("Piirteiden valinta epäonnistui.")

        # Vaihe 4: Kouluta malli
        # KORJATTU KUTSU: Annetaan vain tarvittavat argumentit funktiolle
        model = kouluta_xgboost_malli(X_train, y_train, XGB_PARAMS)
        if model is None: raise ValueError("Mallin koulutus epäonnistui.")

        # Vaihe 5: Evaluoi malli
        xgboost_results = evaluoi_malli(model, X_test, y_test, "Testisetti")
        if xgboost_results is None: raise ValueError("Mallin evaluointi epäonnistui.")

        # Vaihe 5b: Analysoi piirteiden tärkeys
        # Annetaan X_train funktiolle, jotta se saa piirteiden nimet oikein
        analysoi_piirteiden_tarkeys(model, X_train)

        # Vertailu Baselineen (Lisää tähän v0.5 tulokset manuaalisesti)
        print("\nHaetaan baseline-tulokset vertailua varten...")
        # !! MUISTA PÄIVITTÄÄ NÄMÄ ARVOT v0.5 AJON TULOKSISTA !!
        baseline_mae = 12.804  # Esimerkkiarvo v0.5 ajosta
        baseline_rmse = 16.872 # Esimerkkiarvo v0.5 ajosta
        baseline_r2 = 0.331   # Esimerkkiarvo v0.5 ajosta
        if baseline_mae is not None and baseline_rmse is not None and baseline_r2 is not None and xgboost_results is not None:
            print("\n" + "-"*60); print(" Vertailu Baseline-malliin (v0.5)"); print("-"*60)
            print(f"                         Baseline (LR Time Feat) | XGBoost (Weather+AQ+Time Feat)")
            print(f"Mean Absolute Error (MAE):  {baseline_mae:>10.3f}         | {xgboost_results['MAE']:>10.3f}")
            print(f"Root Mean Squared Error (RMSE):{baseline_rmse:>9.3f}         | {xgboost_results['RMSE']:>10.3f}")
            print(f"R-squared (R²):           {baseline_r2:>10.3f}         | {xgboost_results['R2']:>10.3f}")
            print("-"*60); improvement_r2 = xgboost_results['R2'] - baseline_r2; print(f"Parannus R²-arvossa baselineen verrattuna: {improvement_r2:+.3f}")
        else: print("\nBaseline-tuloksia tai XGBoost-tuloksia ei annettu, vertailua ei voida tehdä.")

        # Vaihe 6: Tallenna malli (Valinnainen)
        if SAVE_MODEL and model is not None:
            tallennettu_ok = tallenna_malli(model, MODEL_SAVE_PATH)
            if tallennettu_ok: print(" -> Mallin tallennus onnistui.")
            else: print(" -> Mallin tallennus ei onnistunut.")

    except Exception as e:
        print("\n" + "!"*80); print(f" KRIITTINEN VIRHE PIPELINEN SUORITUKSESSA: {e}");
        import traceback; traceback.print_exc(); print("!"*80)
    finally:
        print("\n" + "="*80); print(" XGBOOST PIPELINE v0.6 SUORITETTU LOPPUUN"); print("="*80)